# Perceptron Learning Algorithm

In [62]:
import numpy as np
import matplotlib.pyplot as plt
from random import uniform,randint
from __future__ import division

In [63]:
#Generates a random line in the square [-1,1], and returns the gradient and intercept
def f ():
    #Generate two random points in the square
    (x1,y1) = (uniform(-1,1),uniform(-1,1))
    (x2,y2) = (uniform(-1,1),uniform(-1,1))
    
    m = (y2 - y1) / (x2 - x1)
    c = y1 - m * x1
    
    return (m, c)

#Define dot product of two vectors (lists)
def dot(K, L):
    if len(K) != len(L):
        return 0

    return sum(i[0] * i[1] for i in zip(K, L))

#Uses PLA to find g, the best estimate of f based on the sample data
def g (x, y):
    w = [0, 0, 0]
    #Number of iterations
    n = 0
    while n < 1E6:
        #create a list of misclassified points
        misclass = []
        #h(x) = sign(w^T.x)
        for i in range(len(x)):
            if np.sign(dot(w, x[i])) != y[i]:
                misclass.append(i)
        n += 1
        #if misclass is empty, we have converged on g
        if not misclass:
            break
        #randomly choose a misclassified point to use to update w
        r = randint(0, len(misclass) - 1)
        #multiply x_i vector by y_i
        adj = [a * y[misclass[r]] for a in x[misclass[r]]]
        #update w
        w[:] = [a + b for a,b in zip(w, adj)]
    return (w,n)

#Generate a list of n lists, containing a point in a square, uniformly distributed in [-1,1]
def nlists(n):
    if n == 0:
        return []
    ls = []
    while n > 0:
        #x_0 = 1
        ls.append([1, uniform(-1,1), uniform(-1,1)])
        n-= 1
    return ls

#for a given x and f, finds the values of y
def ylist(x, (m, c)):
    ys = []
    for i in range(len(x)):
        if x[i][2] > m * x [i][1] + c:
            ys.append(+1)
        else:
            ys.append(-1)
    return ys

def normaliseArea (x_f_top, x_f_bot, x_g_top, x_g_bot):
    """
    Find the area of the rectangle the lines produce when they cross y = +- 1
    """
    norm_area = 1
    
    #Consider 'top' co ordinates
    if x_f_top > 1:
        if x_g_top > x_f_top:
            norm_area += x_g_top - 1
        else:
            norm_area += x_f_top - 1
    elif x_g_top > 1:
        norm_area += x_g_top - 1
    elif x_f_top < -1:
        if x_g_top < x_f_top:
            norm_area += 1 - x_g_top
        else:
            norm_area += 1 - x_f_top
    if x_g_top < -1:
        norm_area += 1 - x_g_top
        
    #Consider 'bottom' co ordinates
    if x_f_bot > 1:
        if x_g_bot > x_f_bot:
            norm_area += x_g_bot - 1
        else:
            norm_area += x_f_bot - 1
    elif x_g_bot > 1:
        norm_area += x_g_bot - 1
    elif x_f_bot < -1:
        if x_g_bot < x_f_bot:
            norm_area += 1 - x_g_bot
        else:
            norm_area += 1 - x_f_bot
    if x_g_bot < -1:
         norm_area += 1 - x_g_bot
            
    #return 2 * norm_area because height is always 2        
    return 2 * norm_area

def probfnotg (w, (m,c)):
    """
    Find the probability a random out of sample point will be misclassified by g
    """
    #First need to transform w to the equation of a line. If w^T * x = 0, x lies on the line of g
    #Find the gradient and intercept of g
    m_g = - w[1] / w[2]
    c_g = - w[0] / w[2]
    
    #Find the points where both lines cross the edges of the unit square
    x_f_top = (1 - c) / m
    x_f_bot = (-1 - c) / m
    x_g_top = (1 - c_g) / m_g
    x_g_bot = (-1 - c_g) / m_g
    
    #Find where the two lines intersect
    x_intersect = (c_g - c) / (m - m_g)
    y_intersect = m * x_intersect + c
    
    #If this lies outside the unit square we have to find area of trapezium
    if x_intersect > 1 or x_intersect < -1 or y_intersect > 1 or y_intersect < -1:
        #Area = h * (a + b) / 2 . Height is always 2 here
        area = (abs(x_g_top - x_f_top) + abs(x_g_bot - x_f_bot))
        #Return normalized value
        return area / normaliseArea(x_f_top, x_f_bot, x_g_top, x_g_bot)
    #Else we find the area of two triangles. Find the length of the bases and the heights
    h_top = 1 - y_intersect
    h_bot = y_intersect + 1
    base_top = abs(x_g_top - x_f_top)
    base_bot = abs(x_g_top - x_f_top)
    #Area = 1/2 * base * height
    area = (0.5 * base_top * h_top) + (0.5 * base_bot * h_bot)
    #Return normalised area
    return area / normaliseArea(x_f_top, x_f_bot, x_g_top, x_g_bot)

In [75]:
#Instantiate the training set, and find g
#To get a reliable estimate for p and n, we repeat 1000 times and average over the values
repeat = 1000
n_sum = 0
p_sum = 0
for _ in range(repeat):
    (m, c) = f()
    x = nlists(10)
    y = ylist(x, (m, c))
    (w, n) = g(x, y)
    p = probfnotg(w, (m,c))
    n_sum += n
    p_sum += p
print (p_sum / repeat, n_sum / repeat)

(0.02662518535623538, 104.338)
